From https://ipython-books.github.io/145-computing-the-voronoi-diagram-of-a-set-of-points/

# Computing the Voronoi diagram of a set of points
## Loading and Calculation part using Pandas & Scipy
The Voronoi diagram of a set of seed points divides space into several regions. Each region contains all points closer to one seed point than to any other seed point.

The Voronoi diagram is a fundamental structure in computational geometry. It is widely used in computer science, robotics, geography, and other disciplines. For example, the Voronoi diagram of a set of metro stations gives us the closest station from any point in the city.

![title](Voronoi_growth_euclidean.gif)

In this recipe, we compute the Voronoi diagram of the set of metro stations in Paris using SciPy.
1.  Let's import the packages:

In [43]:
import numpy as np
import pandas as pd
import scipy.spatial as spatial
import jupytab

2. Let's load the dataset with pandas (which had been obtained on the RATP open data website, the public transport operator in Paris, at http://data.ratp.fr):

In [44]:
df = pd.read_csv('ratp.csv', sep='#', header=None)

df[df.columns[1:]].tail(3)

,1,2,3,4,5
11608,2.350173,48.937238,THEATRE GERARD PHILIPE,SAINT-DENIS,tram
11609,2.301197,48.933118,TIMBAUD,GENNEVILLIERS,tram
11610,2.230144,48.913708,VICTOR BASCH,COLOMBES,tram


3.  The DataFrame object contains the coordinates, name, city, district, and type of station. Let's select all metro stations:

In [45]:
metro = df[(df[5] == 'metro')]

metro[metro.columns[1:]].tail(3)

,1,2,3,4,5
305,2.308041,48.841697,Volontaires,PARIS-15EME,metro
306,2.379884,48.857876,Voltaire (Léon Blum),PARIS-11EME,metro
307,2.304651,48.883874,Wagram,PARIS-17EME,metro


4.  We are going to extract the district number of Paris' stations. With pandas, we can use vectorized string operations using the str attribute of the corresponding column.

In [46]:
# We only extract the district from stations in Paris.
paris = metro[4].str.startswith('PARIS').values

# We create a vector of integers with the district
# number of the corresponding station, or 0 if the
# station is not in Paris.
districts = np.zeros(len(paris), dtype=np.int32)
districts[paris] = metro[4][paris].str.slice(6, 8) \
    .astype(np.int32)
districts[~paris] = 0
ndistricts = districts.max() + 1

5.  We also extract the coordinates of all metro stations:

In [47]:
lon = metro[1]
lat = metro[2]

6.  Now, let's retrieve Paris' map with OpenStreetMap. We specify the map's boundaries with the extreme latitude and longitude coordinates of all our metro stations. We use Smopy to generate the map:

In [48]:
station_df = pd.DataFrame({'district': districts,
             'longitude': lon,
             'latitude': lat}).join(metro[3]).reset_index().rename(columns={'index': 'station_id', 3: 'name'})
station_df.tail(3)

,station_id,district,longitude,latitude,name
305,305,15,2.308041,48.841697,Volontaires
306,306,11,2.379884,48.857876,Voltaire (Léon Blum)
307,307,17,2.304651,48.883874,Wagram


7.  We now compute the Voronoi diagram of the stations using SciPy. A Voronoi object is created with the points coordinates. It contains several attributes we will use for display:

In [49]:
vor = spatial.Voronoi(np.c_[lat, lon])

8.  We create a generic function to display a Voronoi diagram. SciPy already implements such a function, but this function does not take infinite points into account. The implementation we will use is available at http://stackoverflow.com/a/20678647/1595060:

In [50]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """Reconstruct infinite Voronoi regions in a
    2D diagram to finite regions.
    Source:
    [https://stackoverflow.com/a/20678647/1595060](https://stackoverflow.com/a/20678647/1595060)
    """
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")
    new_regions = []
    new_vertices = vor.vertices.tolist()
    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()
    # Construct a map containing all ridges for a
    # given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points,
                                  vor.ridge_vertices):
        all_ridges.setdefault(
            p1, []).append((p2, v1, v2))
        all_ridges.setdefault(
            p2, []).append((p1, v1, v2))
    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        if all(v >= 0 for v in vertices):
            # finite region
            new_regions.append(vertices)
            continue
        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]
        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue
            # Compute the missing endpoint of an
            # infinite ridge
            t = vor.points[p2] - \
                vor.points[p1]  # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal
            midpoint = vor.points[[p1, p2]]. \
                mean(axis=0)
            direction = np.sign(
                np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + \
                direction * radius
            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())
        # Sort region counterclockwise.
        vs = np.asarray([new_vertices[v]
                         for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(
            vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[
            np.argsort(angles)]
        new_regions.append(new_region.tolist())
    return new_regions, np.asarray(new_vertices)

In [51]:
regions, vertices = voronoi_finite_polygons_2d(vor)

## Cleaning Data Frames and preparing for Tableau
We need to clean data and prepare data for Tableau

In [52]:
vertice_df = pd.DataFrame(vertices).reset_index().rename(
    columns={'index': 'vertice_id', 0: 'latitude', 1: 'longitude'}
)
vertice_df.head(3)

,vertice_id,latitude,longitude
0,0,48.757387,2.365479
1,1,48.761133,2.338927
2,2,48.747481,2.417145


In [53]:
points = []
for reg_idx, region in enumerate(regions):
    for vtc_idx, vertice in enumerate(region):
        points.append([reg_idx, vtc_idx, vertice])
        
polygon_df = pd.DataFrame(points).rename(columns={0: 'station_id', 1: 'polygon_id', 2: 'vertice_id'})
polygon_df.head(3)

,station_id,polygon_id,vertice_id
0,0,0,503
1,0,1,444
2,0,2,375


## Jupytab
Creating Tables referential with prepared DataFrame

In [54]:
tables = jupytab.Tables()
tables['stations'] = jupytab.DataFrameTable('List of Paris subway stations', station_df)
tables['vertices'] = jupytab.DataFrameTable('Coordinates of Voronoi cell vertices', vertice_df)
tables['polygons'] = jupytab.DataFrameTable('Polygon definition of Voronoi cells', polygon_df)

Render the Jupytab schema

In [55]:
# GET /schema
tables.render_schema()

[{"id": "stations", "alias": "List of Paris subway stations", "columns": [{"id": "station_id", "dataType": "int"}, {"id": "district", "dataType": "string"}, {"id": "longitude", "dataType": "float"}, {"id": "latitude", "dataType": "float"}, {"id": "name", "dataType": "string"}]}, {"id": "vertices", "alias": "Coordinates of Voronoi cell vertices", "columns": [{"id": "vertice_id", "dataType": "int"}, {"id": "latitude", "dataType": "float"}, {"id": "longitude", "dataType": "float"}]}, {"id": "polygons", "alias": "Polygon definition of Voronoi cells", "columns": [{"id": "station_id", "dataType": "int"}, {"id": "polygon_id", "dataType": "int"}, {"id": "vertice_id", "dataType": "int"}]}]


Render the Jupytab datas

In [56]:
# GET /data
tables.render_data(REQUEST)

NameError: name 'REQUEST' is not defined